In [1]:
import pandas as pd

inference_result_df = pd.read_csv('inference_result_RNN_vanilla_2.csv', usecols=['Predicted_label', 'ReviewerID'])
inference_result_df.head()

,Predicted_label,ReviewerID
0,1,E3pbB5hlBDFr8rxV2ddJYw
1,0,lilz49aurvOvNkCDGrSqIA
2,0,FGMh6kUnR_BYNCq7sgqQnw
3,1,wRGsi90kI0g2ModMB_4RLQ
4,1,Fv0e9RIV9jw5TX3ctA1WbA


In [2]:
inference_result_grouped_df = inference_result_df.groupby('ReviewerID').agg({'Predicted_label': 'value_counts'})
inference_result_grouped_df.head()

Predicted_label
ReviewerID             Predicted_label                 
---1lKK3aKOuomHnwAkAow 1                             99
                       0                             29
---PLwSf5gKdIoVnyRHgBA 1                              2
--0RtXvcOIE4XbErYca6Rw 0                              1
--0VhLFv2XfEKHL0YSCSuw 1                              1

In [3]:
majority_voting_result_ser = inference_result_df.groupby('ReviewerID').max()
majority_voting_result_ser.head()

,Predicted_label
ReviewerID,
---1lKK3aKOuomHnwAkAow,1
---PLwSf5gKdIoVnyRHgBA,1
--0RtXvcOIE4XbErYca6Rw,0
--0VhLFv2XfEKHL0YSCSuw,1
--2DMAd_kGHgbF88H2SLuw,1


In [4]:
majority_voting_result_df = pd.DataFrame(majority_voting_result_ser.Predicted_label).reset_index()

In [5]:
majority_voting_result_df.columns = ['user_id', 'predicted_gender']
majority_voting_result_df.head()

,user_id,predicted_gender
0,---1lKK3aKOuomHnwAkAow,1
1,---PLwSf5gKdIoVnyRHgBA,1
2,--0RtXvcOIE4XbErYca6Rw,0
3,--0VhLFv2XfEKHL0YSCSuw,1
4,--2DMAd_kGHgbF88H2SLuw,1


In [6]:
majority_voting_result_df.predicted_gender.value_counts()

1    442983
0    154766
Name: predicted_gender, dtype: int64

In [7]:
import pandas as pd

disclosed_flag = False
if disclosed_flag:
    disclosed_dataset_df = pd.read_csv('disclosed_dataset.csv')
    mapping = {'female' : 1, 'male' : 0}
    disclosed_dataset_df.replace({'gender': mapping}, inplace=True)
    test_df = disclosed_dataset_df.head()
else:
    undisclosed_csv_df = pd.read_csv('undisclosed_dataset.csv')
    undisclosed_csv_df.head()

In [ ]:
test_df = disclosed_dataset_df.iloc[1:100]

In [ ]:
test_tmp = test_df.groupby('user_id').apply(lambda g: g.gender.mode())
test_tmp.head()

In [ ]:
ser = test_tmp.gender.groupby('user_id').max(level=0)
ser

In [8]:
merged_df = pd.merge(majority_voting_result_df, undisclosed_csv_df, on='user_id', how='outer')

In [9]:
# majority_voting_result_df.count
undisclosed_csv_df.user_id.value_counts()

bLbSNkLggFnqwNNzzq-Ijw    2354
PKEzKWv_FktMm2mGPjwd0Q    1822
QJI9OSEn6ujRCtrX06vs1w    1496
YRcaNlwQ6XXPFDXWtuMGdA    1234
dIIKEfOgo0KqUfGQvGikPg    1198
                          ... 
aCp1HRfU6Y0ZGGUCRZLHWQ       1
GIXFXQcY4VdtCgrpoRci2g       1
gFehGlbM4iAyDIJ1DjQO0g       1
FV7UOdJjG-Mj-YSDrVBVpg       1
NickymmL-exBN-rUvfeyRw       1
Name: user_id, Length: 597765, dtype: int64

In [10]:
undisclosed_csv_df.count()

review_id        2416933
user_id          2416933
business_id      2416936
stars            2416933
date             2416933
useful           2416933
text             2416933
business_name    2416933
user_name        2416933
timestamp        2416933
categories       2416179
gender           2416936
dtype: int64

In [11]:
majority_voting_result_df.count()

user_id             597749
predicted_gender    597749
dtype: int64

In [12]:
inference_result_grouped_df.count()

Predicted_label    770289
dtype: int64

In [13]:
merged_df.count()

user_id             2416933
predicted_gender    2416916
review_id           2416933
business_id         2416936
stars               2416933
date                2416933
useful              2416933
text                2416933
business_name       2416933
user_name           2416933
timestamp           2416933
categories          2416179
gender              2416936
dtype: int64

In [14]:
merged_dropna_df = merged_df.dropna()
merged_dropna_df.count()

user_id             2416159
predicted_gender    2416159
review_id           2416159
business_id         2416159
stars               2416159
date                2416159
useful              2416159
text                2416159
business_name       2416159
user_name           2416159
timestamp           2416159
categories          2416159
gender              2416159
dtype: int64

In [15]:
merged_dropna_df.head()

,user_id,predicted_gender,review_id,business_id,stars,date,useful,text,business_name,user_name,timestamp,categories,gender
0,---1lKK3aKOuomHnwAkAow,1.0,wEJK2PeiS6Au0TOyxTyqUA,Hqs4YNST_ZHbshwyi4bnsQ,5.0,2018-10-11 23:29:57,0.0,The customer service of the owner made me give...,Old School Pizzeria,Monera,1.539293e+09,"Restaurants, Pizza, American (Traditional), It...",unknown
1,---1lKK3aKOuomHnwAkAow,1.0,G2cGRh-mbojvqDl36YkodA,Hqs4YNST_ZHbshwyi4bnsQ,1.0,2018-10-07 15:21:55,0.0,I used to drive from summerlin to go here beca...,Old School Pizzeria,Monera,1.538919e+09,"Restaurants, Pizza, American (Traditional), It...",unknown
2,---1lKK3aKOuomHnwAkAow,1.0,-pk4s5YUD0grEEBt2QYlDA,bPcqucuuClxYrIM8xWoArg,5.0,2011-09-29 20:17:08,4.0,Did anyone mention bacon chocolate ice cream. ...,Due Forni,Monera,1.317320e+09,"Nightlife, Bars, Italian, Pizza, Wine Bars, Re...",unknown
3,---1lKK3aKOuomHnwAkAow,1.0,2AXKTUbIkwuP8wcvzSt7Tg,hubbaEcYPYEZu5Ziz6i0lw,4.0,2011-05-01 15:55:00,1.0,I like the Lamb Tandoori dish they do a lot. I...,Mantra Masala,Monera,1.304258e+09,"Restaurants, Pakistani, Indian",unknown
4,---1lKK3aKOuomHnwAkAow,1.0,3R2e-knpN5lCHu2LVk6hsQ,5aeR9KcboZmhDZlFscnYRA,5.0,2013-01-16 01:28:18,1.0,So Fresh Mama let us host a Homeschool Board G...,Fresh Mama,Monera,1.358296e+09,"Food, Restaurants, Internet Cafes, Juice Bars ...",unknown


In [17]:

# fields = ['business_id','predicted_gender','useful','text', 'stars', 'timestamp']
# merged_dropna_df = merged_dropna_df[fields]
# merged_dropna_df.rename(columns={'predicted_gender': 'gender'})
merged_dropna_df.to_csv('undisclosed_predicted_dataset.csv', index=False)

In [ ]:
fields = ['business_id','predicted_gender','useful','text', 'stars', 'timestamp']
df_male = merged_dropna_df[fields]
df_male.head()